# tf_flowers Dataset
The following `Jupyter Notebook` has been *adapted* from the [Keras blog article](https://blog.keras.io/building-autoencoders-in-keras.html) written by *F. Chollet* on [autoencoders](https://en.wikipedia.org/wiki/Autoencoder), and will focus on training the `Minimal2DAutoencoder` on the `tf_flowers` dataset.

## Setup
Need to get the necessary packages ...

In [ ]:
# check for colab
if "google.colab" in str(get_ipython()):
  # install colab dependencies
  !pip install git+https://github.com/DiogenesAnalytics/autoencoder

## Get tf_flowers Data
The [TensorFlow Flowers](https://www.tensorflow.org/datasets/catalog/tf_flowers) dataset first needs to be downloaded, and then preprocessed.

In [ ]:
# initial libs
import keras
import pathlib

# data location
DATASET_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

# download, get path, and convert to pathlib obj
data_dir = pathlib.Path(
    keras.utils.get_file("flower_photos", origin=DATASET_URL, untar=True, cache_dir="./data/keras")
)

In [ ]:
# use keras util to load raw images into tensorflow.data.Dataset
x_train, x_val = keras.utils.image_dataset_from_directory(
  data_dir,
  color_mode="rgb",
  validation_split=(1 / 7),
  subset="both",
  seed=42,
  image_size=(28, 28),
  batch_size=32,
)

# use a keras layer to normalize
normalization_layer = keras.layers.Rescaling(1./255)

# will apply normalization as well as regrouping each element
def change_inputs(images, labels):
  x = normalization_layer(images)
  return x, x

# apply change
x_train, x_val = x_train.map(change_inputs), x_val.map(change_inputs)

## Autoencoder Training
Finally the *autoencoder* can be trained ...

In [ ]:
# get libs for training ae
from autoencoder.model.minimal import Min2DAE, Min2DParams

# seupt config
config = Min2DParams(
    l0={"input_shape": (28, 28, 3)},
    l2={"units": 32 * 3},
    l3={"units": 28 * 28 * 3},
    l4={"target_shape": (28, 28, 3)},
)

# get ae instance
autoencoder = Min2DAE(config)

# check network topology
autoencoder.summary()

In [ ]:
# get libs for early stopping
from keras.callbacks import EarlyStopping

# create callback early stopping
early_stop_callback = EarlyStopping(monitor="val_loss", patience=3)

# compile ...
autoencoder.compile(optimizer="adam", loss="mse")

# ... and train
history = autoencoder.fit(
    x=x_train,
    validation_data=x_val,
    epochs=50,
    callbacks=[early_stop_callback]
)

In [ ]:
# show training loss
autoencoder.training_history()

## Visualizing Predictions
Now it is possible, using the trained autoencoder, to encode/decode an image and see how it compares to the original ...

In [ ]:
# get viz func
from autoencoder.data import compare_image_predictions

# get samples from validation dataset
val_samples = x_val.take(1)

# get raw numpy arrays
val_input = [item for pair in val_samples.as_numpy_iterator() for item in pair[0]]

# and decoded
decoded_imgs = autoencoder.predict(x=val_samples)

# display
compare_image_predictions(val_input, decoded_imgs)